## Daniel's comparison

In [ ]:
import numpy as np
import pykat
import matplotlib.pyplot as plt

# Functions

Functions for doing the analytics:

In [ ]:
# Speed of light
c = 299792458.0

# Propagation through vacuum
def space(phi, L, W=0):
    return np.exp(-1j*W*L/c)*np.array([[np.cos(phi), -np.sin(phi)],
                                       [np.sin(phi), np.cos(phi)]] )

# Transmission
def trans(t):
    return np.eye(2)*t

# Reflection
def refl(r, neg=False):
    if neg:
        r *= -1
    return np.eye(2)*r

# Nonlinear element
def nle(r, phi):
    '''
    Nonlinear element.
    
    r    - Squeeze exponent. 
    phi  - Squeeze angle.
    '''
    return np.array([[np.cosh(r) + np.sinh(r)*np.cos(2*phi), np.sinh(r)*np.sin(2*phi)], 
                     [np.sinh(r)*np.sin(2*phi), np.cosh(r) - np.sinh(r)*np.cos(2*phi)]])


# Reflection off cavity with internal NLE. 
def r_cav(r1, t1, r2, t2, phi, L, r_sq=0, phi_sq=0, W=0):
    # Must to allow second source through end mirror to be correct for quantum noise. 
    # Thus, currently incorrect for quantum noise calculations if r2 != 1.
    
    N = nle(r_sq, phi_sq)
    R1p = refl(r1)
    R1n = refl(r1, neg=True)
    T1 = trans(t1)
    R2 = refl(r2)
    S = space(phi, L, W)
    
    # Roundtrip.
    M = np.eye(2) - N@S@R2@S@N@R1p
    M = np.linalg.inv(M)
    
    # M_rt = eye(2) - N*S*R2*S*N*R1p
    # M_rt = M_rt.inv()
        
    return R1n + T1@M@N@S@R2@S@N@T1


def nle_in_cav(r1, t1, r2, t2, phi, L, r_sq, phi_sq, P_LO, F):
    '''
    Computes the ASD of quantum noise reflected off a cavity with an internal NLE.
    
    r1      - Input mirror amplitude reflection
    t1      - Input mirror amplitude transmission
    r2      - End mirror amplitude reflection
    t2      - End mirror amplitude transmission
    phi     - Cavity tuning [deg]
    L       - Cavity length [m]
    r_sq    - Squeeze exponent.
    phi_sq  - Squeeze angle [deg]
    P_LO    - Power of the local oscillator [W].
    F       - Frequency offset at which to compute quantum noise [Hz].
    '''
    
    hbar = 1.0545718e-34
    c = 299792458.0

    lam0 = 1.064e-6
    f0 = c/lam0
    w0 = 2.0*np.pi*f0
    W = 2.0*np.pi*F
    
    phi *= np.pi/180.0
    phi_sq *= np.pi/180.0
    

    # Fields
    N = 5
    
    a = 0
    b = 2
    c = 4
    d = 6
    e = 8

    # noise sources (SRC)
    SRC = np.zeros([2*N])
    SRC[a:a+2] = 1
    SRC[e:e+2] = 1

    SRC *= np.sqrt(hbar*w0/c)

    # The noise matrix
    N_Mat = np.eye(2*N, dtype=complex)*-1

    # a to a (source)
    N_Mat[a:a+2, a:a+2] = np.eye(2)

    # a to b
    N_Mat[b:b+2, a:a+2] = r_cav(r1, t1, r2, t2, phi, L, r_sq, phi_sq, W)

    # b to c
    N_Mat[c:c+2, b:b+2] = refl(np.sqrt(2))

    # b to d
    N_Mat[d:d+2, b:b+2] = trans(np.sqrt(2))

    # e to c
    N_Mat[c:c+2, e:e+2] = trans(np.sqrt(2))

    # e to d
    N_Mat[d:d+2, e:e+2] = refl(np.sqrt(2), neg=True)

    # e to e (source)
    N_Mat[e:e+2, e:e+2] = np.eye(2)

    N = np.linalg.inv(N_Mat)

    # Coherent field at PDs
    C = np.sqrt(P_LO)/np.sqrt(2.0)
    D = -np.sqrt(P_LO)/np.sqrt(2.0)

    # ASD readout for both quadratures 
    Q1 = np.sqrt(np.sum(np.abs(N[c,:]*SRC*C - N[d,:]*SRC*D)**2))
    # as above, except LO is at 90 degrees
    Q2 = np.sqrt(np.sum(np.abs(N[c+1,:]*SRC*C - N[d+1,:]*SRC*D)**2))
    
    return Q1, Q2

# Photodetector
def pd(A):
    # A is the complex amplitude
    return np.abs(A)**2

# Amplitude detector 
def ad(A):
    # returns the complex amplitude
    return A[0] - 1j*A[1]

# # Homodyne readout (not finished. Do not use.)
# def HD_readout(M, idx1, idx2, A, a):
#     '''
#     A    - Coherent part of the local oscillator. 
#     a    - Noise part of the local oscillator
#     b    - Noise part to be measured
    
#     Returns
#     --------
#     P_HD   - Coherent power
#     p_hd   - Noise power operator
#     '''
    
#     # From cav
#     # T1 = trans(1.0/np.sqrt(2.0))
#     # R1 = refl(1.0/np.sqrt(2.0))
    
#     # From LO
#     T = trans(1.0/sqrt(2.0))
#     R = refl(1.0/sqrt(2.0), neg=True)
    
#     # Coherent
#     C = T*A
#     D = R*A
    
#     # Readout, coherent
    
#     P_C = C[0]*C[0].conjugate() + C[1]*C[1].conjugate()
#     P_D = D[0]*D[0].conjugate() + D[1]*D[1].conjugate()
    
#     P_HD = P_C - P_D
    
#     # Readout, quantum noise
#     p_c = C[0]*Dagger(c[0]) + C[0].conjugate()*c[0] + C[1]*Dagger(c[1]) + C[1].conjugate()*c[1]
#     p_d = D[0]*Dagger(d[0]) + D[0].conjugate()*d[0] + D[1]*Dagger(d[1]) + D[1].conjugate()*d[1]
    
#     p_hd = p_c - p_d
    
#     return P_HD, p_hd


# # Power Spectral Density. Not Finished. Do not use. 
# def PSD(p):
#     return (p*Dagger(p) + Dagger(p)*p)/2

# Cavity response test

No NLE in the cavity. Just testing the "analytic" equations vs. Finesse.

In [ ]:
# Controller (setting the parameters here)
# -----------------------------------------

# Input laser power
P0 = 2.5
# Input laser phase
phi0 = 0

# Power reflection and transmission coeffiecients 
R1 = 0.81
T1 = 1.0-R1

R2 = 0.9
T2 = 1.0-R2

# NLE-parameters
r_sq = 0
phi_sq = 0

# Tuning of end mirror
phi = 0
# Cavity length
L = 1.0
# Frequency offset
F = 0

## Finesse model

In [ ]:
base = pykat.finesse.kat()
base.Verbose=False
base.parse('''

l laser 1 0 0 n0
s s0 0 n0 nIM1

m IM 0.81 0.19 0 nIM1 nIM2
s sCav 1 nIM2 nEM1
m EM 1 0 0 nEM1 nEM2

pd P nIM1
ad A 0 nIM1

yaxis abs:deg
''')     

In [ ]:
# Finesse run
kat = base.deepcopy()
kat.laser.P = P0
kat.laser.phase = phi0
kat.laser.f = F
kat.A.f = F

kat.IM.R = R1
kat.IM.T = T1
kat.EM.R = R2
kat.EM.T = T2

kat.EM.phi = phi
kat.sCav.L = L

# Sweeping the cavity tuning. Note that positive phi at the end mirror means shorter cavity.
kat.parse('xaxis EM phi lin -90 90 201')

out = kat.run()
P_f = out['P'].real
A_f = out['A']

## Analytics

In [ ]:
# Analytic
# ---------------

# Transforming parameters
phi0 *= np.pi/180

r1 = np.sqrt(R1)
t1 = np.sqrt(T1)
r2 = np.sqrt(R2)
t2 = np.sqrt(T2)

W = 2.0*np.pi*F

# Input field
E1 = np.sqrt(P0)*np.cos(phi0)
E2 = np.sqrt(P0)*np.sin(phi0)
# ... as vector
E = np.array([[E1],[E2]])

# Arrays for storing output power and complex amplitude
P_a = np.zeros(len(out.x))
A_a = np.zeros(len(out.x), dtype=complex)

for k, phi in enumerate(out.x):
    phi *= np.pi/180
    # Matrix describing reflection off cavity
    M = r_cav(r1, t1, r2, t2, -phi, L, r_sq=0, phi_sq=0, W=W)

    # Output field
    E2 = M @ E
    
    # Computing complex amplitude and power. Rotating 180 deg to make it match Finesse. 
    A_a[k] = ad(E2)*np.exp(-1j*np.pi)
    P_a[k] = pd(A_a[k])

# James's analytics

In [ ]:
# from Mathematica, columns are degpsi, reflPower, reflAmp, reflArg
james_analytics = np.genfromtxt("empty_cavity_analytics_versus_tuning.csv", delimiter=',')
james_degpsi, james_power, james_amp, james_arg = (james_analytics[:,0], james_analytics[:,1],
                                                   james_analytics[:,2], james_analytics[:,3])


## Plotting

In [ ]:
# Power
fig = plt.figure(dpi=120, figsize=(9,6))
ax = fig.add_subplot(221)
ax.plot(out.x, P_f, linestyle="-", label='Finesse')
ax.plot(out.x, P_a, linestyle=(0, (5, 10)), label='Analytics')
ax.plot(james_degpsi, james_power, linestyle=(5, (5, 10)), label="James's analytics")

ax.legend(loc="upper right")
ax.grid()
ax.set_yscale('linear')
ax.set_xlabel(r'Cavity tuning [deg]')
ax.set_ylabel(r'Reflected power [W]')
ax.set_title('Frequency offset: {:.1f} MHz'.format(F/1e6))


# Magnitude
ax = fig.add_subplot(222)
ax.plot(out.x, np.abs(A_f), '-', label='Finesse')
ax.plot(out.x, np.abs(A_a), linestyle=(0, (5, 10)), label='Analytics')
ax.plot(james_degpsi, james_amp, linestyle=(5, (5, 10)), label="James's analytics")

ax.legend(loc="upper right")
ax.grid()
ax.set_yscale('linear')
# ax.set_xlabel(r'Cavity tuning [deg]')
ax.set_ylabel(r'Reflected magnitude [$\sqrt{\mathrm{W}}$]')


# Phase
ax = fig.add_subplot(224)
ax.plot(out.x, np.angle(A_f), linestyle="-", label='Finesse')
ax.plot(out.x, np.angle(A_a), linestyle=(0, (7, 8)), label='Analytics')
ax.plot(james_degpsi, james_arg, linestyle=(5, (5, 10)), label="James's analytics")

ax.legend(loc="upper right")
ax.grid()
ax.set_yscale('linear')
ax.set_xlabel(r'Cavity tuning [deg]')
ax.set_ylabel(r'Phase [rad]')

plt.savefig("empty_cavity_versus_tuning_comparison.pdf", bbox='tight')
plt.show(fig)

# Crystal in cavity

All parameters for the analytics are extracted from Finesse. Thus, to change parameters, alter the Finesse model below.

## Base Finesse model

In [ ]:
# Single-trip squeeze dB
x_sq = 0.01
# Squeeze angle
phi_sq = 0

base = pykat.finesse.kat()
base.parse('''

l laser 1 0 0 n0
s s0 0 n0 nBS4

dbs FI nFIa nFIb nFIc nFId

s s1 0 nFIc nIM1

m IM 0.81 0.19 0 nIM1 nIM2
s sCav1 0 nIM2 nNLE1
nle NLE {0} {1} nNLE1 nNLE2
s sCav 1 nNLE2 nEM1
m EM 1 0 0.01 nEM1 nEM2

s s2 0 nFId nBS1

bs BS 0.5 0.5 0 0 nBS1 nBS2 nBS3 nBS4

hd HD 180 nBS2 nBS3
qhd QHD 180 nBS2 nBS3

pd P1 nBS2
pd P2 nBS3

fsig sig 1

yaxis abs
'''.format(x_sq, phi_sq))

DBS

1 -- > 3

2 ---> 1

3 ---> 4

4 ---> 2

In [ ]:
# Extracting parameters that will stay the same from the Finesse-model 
r1 = np.sqrt(base.IM.R.value)
t1 = np.sqrt(base.IM.T.value)
r2 = np.sqrt(base.EM.R.value)
t2 = np.sqrt(base.EM.T.value)
L = base.sCav.L.value

# r_sq = np.log(10**(x_sq/10.0))

## Sweeping squeeze magnitude with cavity on resonance

In [ ]:
# Adding xaxis and running Finesse.
kat = base.deepcopy()
kat.noxaxis = False
kat.parse('xaxis NLE r lin -1 1 400')
out = kat.run()

# Extracting some more parameters
F = kat.signals.f.value
P_LO = kat.laser.P.value
phi_LO = kat.laser.phase.value
phi = kat.IM.phi.value - kat.EM.phi.value

In [ ]:
# Running analytics
# ------------------
# Converting from squeeze magnitude in dB (Finesse) to squeeze exponent r (analytics). 
# Note that we use 10 instead of 20. Thus, using formula for power dB even though we want this in amplitude dB.
r_sqs = np.log(10**(out.x/10.0))

Q1 = np.zeros(len(out.x))
Q2 = np.zeros(len(out.x))

for k, r_sq in enumerate(r_sqs):
    Q1[k], Q2[k] = nle_in_cav(r1, t1, r2, t2, phi, L, r_sq, phi_sq, P_LO, F)

In [ ]:
# Plotting

fig = plt.figure(dpi=120)
ax = fig.add_subplot(111)
ax.plot(out.x, out['QHD'], label='Finesse')
# Note that the extra factor of sqrt(2) is needed to make a match.
ax.plot(out.x, Q1*np.sqrt(2),'--', label='Analytic')

ax.legend()
ax.grid()
ax.set_yscale('log')
ax.set_xlabel(r'Single-pass squeeze magnitude [dB]')
ax.set_ylabel(r'Quantum noise [W/$\sqrt{\mathrm{Hz}}$]')
ax.set_title('Frequency: {:.1e} Hz. Cavity detuning: {} deg'.format(F, phi))
plt.show(fig)

# Sweeping cavity tuning

With frequency and squeeze magnitude set to arbitrary values.

In [ ]:
kat = base.deepcopy()
kat.noxaxis = False
kat.parse('xaxis EM phi lin -10 10 400')
out = kat.run()

# Extracting parameters
F = kat.signals.f.value
P_LO = kat.laser.P.value
phi_LO = kat.laser.phase.value
r_sq = np.log(10**(x_sq/10.0))


In [ ]:
# Running analytics
phis = -out.x
Q1 = np.zeros(len(out.x))
Q2 = np.zeros(len(out.x))

# ASDs for the two quadratures. Looping over the different tunings used in the Finesse run
for k, phi in enumerate(out.x):
    # Note that we set -phi, as positive phi gives shorter cavity in Finesse, while it 
    # gives longer cavity in the analytics. 
    Q1[k], Q2[k] = nle_in_cav(r1, t1, r2, t2, -phi, L, r_sq, phi_sq, P_LO, F)

In [ ]:
fig = plt.figure(dpi=120)
ax = fig.add_subplot(111)
ax.plot(out.x, out['QHD'], label='Finesse')
ax.plot(out.x, Q1*np.sqrt(2),'--', label='Analytic')

ax.legend()
ax.grid()
ax.set_yscale('log')
ax.set_xlabel(r'Cavity tuning [deg]')
ax.set_ylabel(r'Quantum noise [W/$\sqrt{\mathrm{Hz}}$]')
ax.set_title('Frequency: {:.1e} Hz. SQZ magnitude: {} dB'.format(F, x_sq))
plt.show(fig)